<h1>Serial Ports</h1>
<hr style="border: 1px solid #000;">
<span>
<h2>Serial Port abstraction for ATT.</h2>
</span>
<br>
<span>
This notebook shows the ATT Serial Port abstraction module.<br>
This module was created for enabling testing on ATT framework.
The Serial Port abstraction provides an Abstract base class so it can be extended and implement whatever kind of serial port we need.
We have used this class hierarchy to build some Mocks, in order to test the ATT framework.
</span>


Set modules path first:

In [12]:
import sys
sys.path.insert(0, '/home/asanso/workspace/att-spyder/att/src/python/')

The main abstract base class is the following one:

class SerialPort:
	__metaclass__ = abc.ABCMeta
	
	@abc.abstractmethod
	def isOpen(self):
		pass
		
	@abc.abstractmethod
	def readline(self):
		pass
		
	@abc.abstractmethod
	def close(self):
		pass
	
	@abc.abstractmethod
	def get_port(self):
		return ""
		
	@abc.abstractmethod
	def get_baudrate(self):
		return 0

As an example, we can see a dummy implementation:

class DummySerialPort (SerialPort):
	def __init__(self, port = None, baud = None):
		pass
	
	def isOpen(self):
		return True
		
	def close(self):
		pass
		
	def get_port(self):
		return ""
		
	def get_baudrate(self):
		return 0
		
	def readline(self):
		time_delay = int(3*random.random())+1
		time.sleep(time_delay)
		return self.gen_random_line()
		
	def gen_random_line(self):
		return "Hee"

<h2>Building Serial Ports</h2>

<span>
In order to build an instance of a SerialPort class, we have 2 options:
<ul>
<li>Call the constructor directly</li>
<li>Use a Builder</li>
</ul>
</span>


<h3>Calling the constructor</h3>

In [22]:
import hit.serial.serial_port

port=""
baud=0
dummySerialPort = hit.serial.serial_port.DummySerialPort(port, baud)

<span>
The DummSerialPort is very simple. It just says "Hee" (after a few seconds) when its method "readline()" is called.<br>
Port and Baud are useless here.
</span>

In [23]:
print dummySerialPort.readline()

Hee


<span>
Let's create a more interesting Serialport instance,
</span>

In [24]:
import hit.serial.serial_port

port=""
baud=0
emulatedSerialPort = hit.serial.serial_port.ATTEmulatedSerialPort(port, baud)

<span>
The ATTEmulatedSerialPort will emulate a real ATT serial port reading.<br>
Port and Baud are useless here.
</span>

In [25]:
print emulatedSerialPort.readline()

hit: {1636 0 1104 2928 652 460 568 1 r}


<h3>Using a Builder</h3>

<span>
Let's use a builder now.
</span>
<span>
We can choose the builder we want and build as many SerialPorts we want.
</span>

In [26]:
import hit.serial.serial_port_builder

builder = hit.serial.serial_port_builder.ATTEmulatedSerialPortBuilder()

port=""
baud=0

emulatedSerialPort1 = builder.build_serial_port(port, baud)
emulatedSerialPort2 = builder.build_serial_port(port, baud)
emulatedSerialPort3 = builder.build_serial_port(port, baud)
emulatedSerialPort4 = builder.build_serial_port(port, baud)
emulatedSerialPort5 = builder.build_serial_port(port, baud)
emulatedSerialPort6 = builder.build_serial_port(port, baud)
emulatedSerialPort7 = builder.build_serial_port(port, baud)

<span>
And call "readline()"
</span>

In [27]:
print emulatedSerialPort5.readline()

hit: {0 2080 3824 3132 884 1812 1088 1 r}


<span>
There is a special Serial port abstraction that is fed from a file.<br>
This is useful when we want to "mock" the serial port and give it previously stored readings.
</span>
<span>
This is interesting, for example, in order to reproduce, or visualize the repetition of an interesting set of hits in a game. Because Serial line is Real-Time, there are situations where it is needed to provide the ATT framework with a set of know hits, previously stored.
</span>


<span>
We can use the data use in "Train points importer".
</span>

In [28]:
!head -10 train_points_import_data/arduino_raw_data.txt

(6,6)
hit: {0:25 1549:4 2757:4 1392:4 2264:7 1764:7 1942:5 2984:5 r}
hit: {0:33 1521:6 2712:4 1364:4 2226:10 1894:10 1905:8 2932:5 r}
hit: {0:34 1554:7 2766:5 1233:4 2273:10 1766:4 1951:12 2993:4 r}
hit: {0:31 1667:6 2878:4 1345:4 2209:4 1880:8 2056:14 2935:4 r}
hit: {0:28 1529:6 2737:5 1211:5 2244:9 1735:4 1920:6 2967:4 r}
hit: {0:35 1525:8 2720:5 1207:6 2237:10 1744:5 1922:5 2939:5 r}
hit: {0:9 1521:10 2746:5 1218:8 2251:9 1744:5 1929:4 2971:4 r}
hit: {0:16 1694:6 2910:5 1372:4 2415:8 1913:6 2098:10 2965:4 r}
hit: {0:5 1703:6 2911:5 1561:4 2416:9 1402:4 2094:6 3136:5 r}


In [29]:
import hit.serial.serial_port_builder

builder = hit.serial.serial_port_builder.ATTHitsFromFilePortBuilder()

port="train_points_import_data/arduino_raw_data.txt"
baud=0

fileSerialPort = builder.build_serial_port(port, baud)

<span>
And now we will read some lines:
</span>

In [31]:
for i in range(20):
    print fileSerialPort.readline()

hit: {298:16 1332:5 1519:4 0:12 1186:4 187:19 525:11 2913:6 r}
hit: {137:12 311:5 1492:5 0:13 1170:5 190:21 530:12 2906:8 r}
hit: {298:17 1491:5 1503:5 0:11 1342:6 188:18 523:8 2898:4 r}
hit: {0:12 372:4 1401:5 34:15 1255:7 57:21 595:20 2485:7 r}
hit: {148:9 340:4 1379:4 0:10 1220:4 204:15 568:18 2963:6 r}
hit: {146:4 1347:4 1537:4 0:9 1380:6 202:17 550:8 2978:6 r}
hit: {321:16 332:4 2044:8 0:12 1224:5 196:19 567:12 2958:8 r}
hit: {319:12 1362:7 1543:5 0:13 1228:5 194:21 565:18 2964:8 r}
hit: {992:5 1525:4 3046:6 842:10 1886:4 0:8 876:10 2755:4 r}
hit: {965:5 1323:4 3077:9 661:5 1866:5 0:7 695:11 2786:4 r}
hit: {973:6 1502:5 3236:8 1007:8 2045:5 0:6 858:23 2081:4 r}
hit: {963:6 1477:5 3161:6 815:7 1994:5 0:7 847:14 3390:4 r}
hit: {957:7 1473:6 3170:8 824:8 2018:6 0:9 856:15 3399:5 r}
hit: {943:4 1628:4 3189:7 972:10 1991:4 0:4 841:18 2897:4 r}
hit: {970:6 1489:5 3050:4 828:6 2046:6 0:4 860:25 2082:4 r}
hit: {966:7 1486:6 3033:6 818:8 2041:5 0:6 850:26 2077:4 r}
hit: {977:5 1503:7 3027: